In [ ]:
arcpy.management.GenerateTessellation(
    Output_Feature_Class=r"C:\Users\hanna\OneDrive\Desktop\Washington Pollutants\Default.gdb\Wash_Tessellation",
    Extent='-13685241.3616923 5920011.72187122 -13541887.2332506 6178888.75456167 PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
    Shape_Type="HEXAGON",
    Size="25 SquareMiles",
    Spatial_Reference='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]];-20037700 -30241100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision',
    H3_Resolution=7
)

In [ ]:
arcpy.analysis.Clip(
    in_features="Heavy_Traffic_Prox",
    clip_features="Wash_Tessellation",
    out_feature_class=r"C:\Users\hanna\OneDrive\Desktop\Washington Pollutants\Default.gdb\Traffic_Clip",
    cluster_tolerance=None
)

In [ ]:
arcpy.analysis.Clip(
    in_features="Toxic_Releases_from_Facilities_(RSEI_Model)",
    clip_features="Wash_Tessellation",
    out_feature_class=r"C:\Users\hanna\OneDrive\Desktop\Washington Pollutants\Default.gdb\Toxic_Clip",
    cluster_tolerance=None
)

In [ ]:
arcpy.analysis.Clip(
    in_features="Proximity_to_Hazardous_Waste",
    clip_features="Wash_Tessellation",
    out_feature_class=r"C:\Users\hanna\OneDrive\Desktop\Washington Pollutants\Default.gdb\HazWaste_Clip",
    cluster_tolerance=None
)

In [ ]:
arcpy.analysis.Union(
    in_features="Traffic_Clip #;HazWaste_Clip #;Toxic_Clip #",
    out_feature_class=r"C:\Users\hanna\OneDrive\Desktop\Washington Pollutants\Default.gdb\Traffic_Clip_Union",
    join_attributes="ALL",
    cluster_tolerance=None,
    gaps="GAPS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table="Traffic_Clip_Union",
    field="RSEI_Float",
    expression="float(!RSEI_Normalized!)",
    expression_type="PYTHON3",
    code_block="""def safe_float(x):
    try:
        return float(x)
    except:
        return None""",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)


In [ ]:
arcpy.management.CalculateField(
    in_table="Traffic_Clip_Union",
    field="Traffic_Float",
    expression="float(!Normalized_Traffic!)",
    expression_type="PYTHON3",
    code_block="""def safe_float(x):
    try:
        return float(x)
    except:
        return None""",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table="Traffic_Clip_Union",
    field="Waste_Float",
    expression="float(!Normalized_Waste!)",
    expression_type="PYTHON3",
    code_block="""def safe_float(x):
    try:
        return float(x)
    except:
        return None""",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.management.CalculateField(
    in_table="Pollutant_Union",
    field="Cumulative_Exposure_Index",
    expression="float((!RSEI_Float! + !Waste_Float! + !Traffic_Float!)/3)",
    expression_type="PYTHON3",
    code_block="",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
arcpy.analysis.SpatialJoin(
    target_features="Env Health Disparities",
    join_features="Pollutant_Union",
    out_feature_class=r"C:\Users\hanna\OneDrive\Desktop\Washington Pollutants\Default.gdb\Heavy_Traffic_Pr_SpatialJoin",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='Census_Tract "Census_Tract" true true false 0 BigInteger 0 0,First,#,Env Health Disparities,Census_Tract,-1,-1;EHD_Rank "EHD_Rank" true true false 0 Short 0 0,First,#,Env Health Disparities,EHD_Rank,-1,-1;Shape__Area "Shape__Area" false true true 0 Double 0 0,First,#,Env Health Disparities,Shape__Area,-1,-1;Shape__Length "Shape__Length" false true true 0 Double 0 0,First,#,Env Health Disparities,Shape__Length,-1,-1;FID_Traffic_Clip "FID_Traffic_Clip" true true false 4 Long 0 0,First,#,Pollutant_Union,FID_Traffic_Clip,-1,-1;Census_Tract_1 "Census_Tract" true true false 8 BigInteger 0 0,First,#,Pollutant_Union,Census_Tract,-1,-1;IBL_Rank "IBL_Rank" true true false 2 Short 0 0,First,#,Pollutant_Union,IBL_Rank,-1,-1;Env_Exp_Rank "Env_Exp_Rank" true true false 2 Short 0 0,First,#,Pollutant_Union,Env_Exp_Rank,-1,-1;EHD_Rank_1 "EHD_Rank" true true false 2 Short 0 0,First,#,Pollutant_Union,EHD_Rank,-1,-1;Proximity_to_Heavy_Traffic_Road "Proximity_to_Heavy_Traffic_Road" true true false 4 Float 0 0,First,#,Pollutant_Union,Proximity_to_Heavy_Traffic_Road,-1,-1;FID_Toxic_Clip "FID_Toxic_Clip" true true false 4 Long 0 0,First,#,Pollutant_Union,FID_Toxic_Clip,-1,-1;Census_Tract_12 "Census_Tract" true true false 8 BigInteger 0 0,First,#,Pollutant_Union,Census_Tract_1,-1,-1;Average_RSEI_Concentrations "Average_RSEI_Concentrations" true true false 4 Float 0 0,First,#,Pollutant_Union,Average_RSEI_Concentrations,-1,-1;IBL_Rank_1 "IBL_Rank" true true false 2 Short 0 0,First,#,Pollutant_Union,IBL_Rank_1,-1,-1;Env_Exp_Rank_1 "Env_Exp_Rank" true true false 2 Short 0 0,First,#,Pollutant_Union,Env_Exp_Rank_1,-1,-1;EHD_Rank_12 "EHD_Rank" true true false 2 Short 0 0,First,#,Pollutant_Union,EHD_Rank_1,-1,-1;FID_HazWaste_Clip "FID_HazWaste_Clip" true true false 4 Long 0 0,First,#,Pollutant_Union,FID_HazWaste_Clip,-1,-1;Census_Tract_12_13 "Census_Tract" true true false 8 BigInteger 0 0,First,#,Pollutant_Union,Census_Tract_12,-1,-1;IBL_Rank_12 "IBL_Rank" true true false 2 Short 0 0,First,#,Pollutant_Union,IBL_Rank_12,-1,-1;EHD_Rank_12_13 "EHD_Rank" true true false 2 Short 0 0,First,#,Pollutant_Union,EHD_Rank_12,-1,-1;Env_Eff_Rank "Env_Eff_Rank" true true false 2 Short 0 0,First,#,Pollutant_Union,Env_Eff_Rank,-1,-1;Average_PTSDF "Average_PTSDF" true true false 4 Float 0 0,First,#,Pollutant_Union,Average_PTSDF,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,Pollutant_Union,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,Pollutant_Union,Shape_Area,-1,-1;RSEI_Normalized "RSEI_Normalized" true true false 512 Text 0 0,First,#,Pollutant_Union,RSEI_Normalized,0,511;Normalized_Waste "Normalized_Waste" true true false 512 Text 0 0,First,#,Pollutant_Union,Normalized_Waste,0,511;Normalized_Traffic "Normalized_Traffic" true true false 512 Text 0 0,First,#,Pollutant_Union,Normalized_Traffic,0,511;RSEI_Float "RSEI_Float" true true false 4 Float 0 0,First,#,Pollutant_Union,RSEI_Float,-1,-1;Waste_Float "Waste_Float" true true false 4 Float 0 0,First,#,Pollutant_Union,Waste_Float,-1,-1;Traffic_Float "Traffic_Float" true true false 4 Float 0 0,First,#,Pollutant_Union,Traffic_Float,-1,-1;Total_exposure "Total_exposure" true true false 512 Text 0 0,First,#,Pollutant_Union,Total_exposure,0,511;Total_exposure_float "Total_exposure_float" true true false 8 Double 0 0,First,#,Pollutant_Union,Total_exposure_float,-1,-1;Cumulative_Exposure_Index "Cumulative_Exposure_Index" true true false 4 Float 0 0,First,#,Pollutant_Union,Cumulative_Exposure_Index,-1,-1',
    match_option="INTERSECT",
    search_radius=None,
    distance_field_name="",
    match_fields=None
)

In [ ]:
arcpy.analysis.SummarizeWithin(
    in_polygons="Pollutant_Union",
    in_sum_features="Env Health Disparities",
    out_feature_class=r"C:\Users\hanna\OneDrive\Desktop\Washington Pollutants\Default.gdb\EnvHealthDisparities_SummarizeWithin",
    keep_all_polygons="KEEP_ALL",
    sum_fields="EHD_Rank Mean",
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)